예측값 사용 VS 사용하지 않았을 때의 비교.

In [4]:
## install required packages
!pip install swig
!pip install wrds
!pip install pyportfolioopt
## install finrl library
!pip install -q condacolab
!apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

  Using cached pyportfolioopt-1.5.5-py3-none-any.whl (61 kB)
     -------------------------------------- 892.6/892.6 kB 2.8 MB/s eta 0:00:00
  Using cached scs-3.2.3-cp310-cp310-win_amd64.whl (8.2 MB)
     -------------------------------------- 292.9/292.9 kB 1.1 MB/s eta 0:00:00
  Using cached ecos-2.0.12-cp310-cp310-win_amd64.whl (72 kB)
  Using cached qdldl-0.1.7-cp310-cp310-win_amd64.whl (83 kB)


'apt-get'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to c:\users\public\documents\estsoft\creatortemp\pip-req-build-2wreq9ph
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit cfdbe462627bd6355dccee215c1aac65873c6d67
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to c:\users\public\documents\estsoft\creatortemp\pip-install-qru5k456\elegantrl_35eccca952f24aa4a66b05925c0b0253
  Resolved https://github.com/AI4Finance-Foundation/ElegantRL.git to commit 59fe4b4a1cca0a28e8a5fa4fb80eed9b5d472978
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ----------------------

  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git 'C:\Users\Public\Documents\ESTsoft\CreatorTemp\pip-req-build-2wreq9ph'
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git 'C:\Users\Public\Documents\ESTsoft\CreatorTemp\pip-install-qru5k456\elegantrl_35eccca952f24aa4a66b05925c0b0253'


In [5]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL")

import itertools

c:\Users\se99a\anaconda3\lib\site-packages\pyfolio\pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [6]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])



In [7]:
# from config.py, TRAIN_START_DATE is a string
TRAIN_START_DATE
# from config.py, TRAIN_END_DATE is a string
TRAIN_END_DATE

'2020-07-31'

In [8]:
TRAIN_START_DATE = '2010-01-01'
TRAIN_END_DATE = '2021-10-01'
TRADE_START_DATE = '2021-10-01'
TRADE_END_DATE = '2023-05-05'

In [9]:
tickers = ['XLB','XLE','XLF','XLI','XLK','XLP','XLU','XLV','XLY']

In [10]:
df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = tickers).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (30213, 8)


In [11]:
df.sort_values(['date','tic'],ignore_index=True)

,date,open,high,low,close,volume,tic,day
0,2010-01-04,33.580002,34.020000,33.450001,25.364519,7567500,XLB,0
1,2010-01-04,57.919998,58.810001,57.790001,37.747696,16928400,XLE,0
2,2010-01-04,11.795288,11.965881,11.770918,9.344018,92942347,XLF,0
3,2010-01-04,28.090000,28.320000,27.959999,21.793045,7471500,XLI,0
4,2010-01-04,23.139999,23.290001,23.100000,19.110191,8449400,XLK,0
...,...,...,...,...,...,...,...,...
30208,2023-05-04,147.660004,148.539993,146.929993,147.231598,5430500,XLK,3
30209,2023-05-04,76.739998,76.860001,76.220001,75.931320,11682600,XLP,3
30210,2023-05-04,68.190002,68.860001,67.550003,68.042702,14631100,XLU,3
30211,2023-05-04,132.899994,132.979996,131.839996,131.857208,9625300,XLV,3


In [12]:
INDICATORS

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

In [13]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = INDICATORS,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (3356, 8)
Successfully added vix
Successfully added turbulence index


In [14]:
processed

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2010-01-04,33.580002,34.020000,33.450001,25.364519,7567500,XLB,0,0.000000,25.521512,25.289541,100.000000,66.666667,100.000000,25.364519,25.364519,20.040001,0.000000
1,2010-01-04,57.919998,58.810001,57.790001,37.747696,16928400,XLE,0,0.000000,25.521512,25.289541,100.000000,66.666667,100.000000,37.747696,37.747696,20.040001,0.000000
2,2010-01-04,11.795288,11.965881,11.770918,9.344018,92942347,XLF,0,0.000000,25.521512,25.289541,100.000000,66.666667,100.000000,9.344018,9.344018,20.040001,0.000000
3,2010-01-04,28.090000,28.320000,27.959999,21.793045,7471500,XLI,0,0.000000,25.521512,25.289541,100.000000,66.666667,100.000000,21.793045,21.793045,20.040001,0.000000
4,2010-01-04,23.139999,23.290001,23.100000,19.110191,8449400,XLK,0,0.000000,25.521512,25.289541,100.000000,66.666667,100.000000,19.110191,19.110191,20.040001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30199,2023-05-03,149.690002,150.660004,148.179993,147.940125,5815100,XLK,2,1.101265,151.344400,144.908068,55.182961,46.442993,6.225843,147.699630,143.517544,18.340000,4.219994
30200,2023-05-03,77.059998,77.379997,76.639999,76.119987,11656600,XLP,2,0.875702,77.189108,74.079747,58.998054,93.547472,22.508083,74.795711,73.269686,18.340000,4.219994
30201,2023-05-03,68.639999,69.050003,67.989998,67.526855,11857200,XLU,2,0.319781,69.807814,67.590043,50.161291,8.126302,8.132944,67.692193,66.656867,18.340000,4.219994
30202,2023-05-03,134.259995,134.899994,133.270004,132.912888,8961500,XLV,2,0.915705,134.927106,131.520086,53.906948,54.096702,6.687990,131.318106,129.596837,18.340000,4.219994


In [15]:
processed.sort_values(['date','tic'],ignore_index=True).head()

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2010-01-04,33.580002,34.020000,33.450001,25.364519,7567500,XLB,0,0.0,25.521512,25.289541,100.0,66.666667,100.0,25.364519,25.364519,20.040001,0.0
1,2010-01-04,57.919998,58.810001,57.790001,37.747696,16928400,XLE,0,0.0,25.521512,25.289541,100.0,66.666667,100.0,37.747696,37.747696,20.040001,0.0
2,2010-01-04,11.795288,11.965881,11.770918,9.344018,92942347,XLF,0,0.0,25.521512,25.289541,100.0,66.666667,100.0,9.344018,9.344018,20.040001,0.0
3,2010-01-04,28.090000,28.320000,27.959999,21.793045,7471500,XLI,0,0.0,25.521512,25.289541,100.0,66.666667,100.0,21.793045,21.793045,20.040001,0.0
4,2010-01-04,23.139999,23.290001,23.100000,19.110191,8449400,XLK,0,0.0,25.521512,25.289541,100.0,66.666667,100.0,19.110191,19.110191,20.040001,0.0


In [16]:
list_ticker = processed["tic"].unique().tolist() #ticker 리스트 불러오기
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str)) #전체 데이터 날짜 날짜 리스트
combination = list(itertools.product(list_date,list_ticker)) #date, ticker 의 combination

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left") #date, tic combination 기준으로 우측에 해당되는 정보들 정리
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)
processed_full

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2010-01-04,XLB,33.580002,34.020000,33.450001,25.364519,7567500.0,0.0,0.000000,25.521512,25.289541,100.000000,66.666667,100.000000,25.364519,25.364519,20.040001,0.000000
1,2010-01-04,XLE,57.919998,58.810001,57.790001,37.747696,16928400.0,0.0,0.000000,25.521512,25.289541,100.000000,66.666667,100.000000,37.747696,37.747696,20.040001,0.000000
2,2010-01-04,XLF,11.795288,11.965881,11.770918,9.344018,92942347.0,0.0,0.000000,25.521512,25.289541,100.000000,66.666667,100.000000,9.344018,9.344018,20.040001,0.000000
3,2010-01-04,XLI,28.090000,28.320000,27.959999,21.793045,7471500.0,0.0,0.000000,25.521512,25.289541,100.000000,66.666667,100.000000,21.793045,21.793045,20.040001,0.000000
4,2010-01-04,XLK,23.139999,23.290001,23.100000,19.110191,8449400.0,0.0,0.000000,25.521512,25.289541,100.000000,66.666667,100.000000,19.110191,19.110191,20.040001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43807,2023-05-03,XLK,149.690002,150.660004,148.179993,147.940125,5815100.0,2.0,1.101265,151.344400,144.908068,55.182961,46.442993,6.225843,147.699630,143.517544,18.340000,4.219994
43808,2023-05-03,XLP,77.059998,77.379997,76.639999,76.119987,11656600.0,2.0,0.875702,77.189108,74.079747,58.998054,93.547472,22.508083,74.795711,73.269686,18.340000,4.219994
43809,2023-05-03,XLU,68.639999,69.050003,67.989998,67.526855,11857200.0,2.0,0.319781,69.807814,67.590043,50.161291,8.126302,8.132944,67.692193,66.656867,18.340000,4.219994
43810,2023-05-03,XLV,134.259995,134.899994,133.270004,132.912888,8961500.0,2.0,0.915705,134.927106,131.520086,53.906948,54.096702,6.687990,131.318106,129.596837,18.340000,4.219994


In [17]:
#prediction 업로드시 확인 필요부분
prediction_5 = pd.read_csv('prediction_5.csv').set_index('Unnamed: 0')
prediction_10 = pd.read_csv('prediction_10.csv').set_index('Unnamed: 0')
prediction_30 = pd.read_csv('prediction_30.csv').set_index('Unnamed: 0')

In [18]:
#prediction 업로드시 확인 필요부분


prediction_5_list = []
for i in range(len(prediction_5)):
    for j in range(len(tickers)):
        prediction_5_list.append(prediction_5.iloc[i,j])
        
for i in range((len(processed_full)-len(prediction_5_list))//len(tickers)):
    for j in range(len(tickers)):
        prediction_5_list.append(prediction_5.iloc[-1,j])

prediction_10_list = []
for i in range(len(prediction_10)):
    for j in range(len(tickers)):
        prediction_10_list.append(prediction_10.iloc[i,j])

for i in range((len(processed_full)-len(prediction_10_list))//len(tickers)):
    for j in range(len(tickers)):
        prediction_10_list.append(prediction_10.iloc[-1,j])
    
prediction_30_list = []
for i in range(len(prediction_30)):
    for j in range(len(tickers)):
        prediction_30_list.append(prediction_30.iloc[i,j])
for i in range((len(processed_full)-len(prediction_30_list))//len(tickers)):
    for j in range(len(tickers)):
        prediction_30_list.append(prediction_30.iloc[-1,j])
        
print(len(prediction_5_list))

30204


In [19]:
#prediction 업로드시 확인 필요부분

processed_full['prediction_5']=prediction_5_list
processed_full['prediction_10']=prediction_10_list
processed_full['prediction_30']=prediction_30_list

processed_full

,date,tic,open,high,low,close,volume,day,macd,boll_ub,...,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence,prediction_5,prediction_10,prediction_30
0,2010-01-04,XLB,33.580002,34.020000,33.450001,25.364519,7567500.0,0.0,0.000000,25.521512,...,100.000000,66.666667,100.000000,25.364519,25.364519,20.040001,0.000000,25.001299,25.008287,28.824316
1,2010-01-04,XLE,57.919998,58.810001,57.790001,37.747696,16928400.0,0.0,0.000000,25.521512,...,100.000000,66.666667,100.000000,37.747696,37.747696,20.040001,0.000000,43.503914,40.740055,41.319206
2,2010-01-04,XLF,11.795288,11.965881,11.770918,9.344018,92942347.0,0.0,0.000000,25.521512,...,100.000000,66.666667,100.000000,9.344018,9.344018,20.040001,0.000000,9.368776,8.849185,10.953069
3,2010-01-04,XLI,28.090000,28.320000,27.959999,21.793045,7471500.0,0.0,0.000000,25.521512,...,100.000000,66.666667,100.000000,21.793045,21.793045,20.040001,0.000000,25.687847,26.830729,29.474950
4,2010-01-04,XLK,23.139999,23.290001,23.100000,19.110191,8449400.0,0.0,0.000000,25.521512,...,100.000000,66.666667,100.000000,19.110191,19.110191,20.040001,0.000000,23.967155,25.295761,25.559303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43807,2023-05-03,XLK,149.690002,150.660004,148.179993,147.940125,5815100.0,2.0,1.101265,151.344400,...,55.182961,46.442993,6.225843,147.699630,143.517544,18.340000,4.219994,159.765030,199.236300,229.869460
43808,2023-05-03,XLP,77.059998,77.379997,76.639999,76.119987,11656600.0,2.0,0.875702,77.189108,...,58.998054,93.547472,22.508083,74.795711,73.269686,18.340000,4.219994,77.748860,81.883970,84.795660
43809,2023-05-03,XLU,68.639999,69.050003,67.989998,67.526855,11857200.0,2.0,0.319781,69.807814,...,50.161291,8.126302,8.132944,67.692193,66.656867,18.340000,4.219994,72.211586,72.498500,82.259070
43810,2023-05-03,XLV,134.259995,134.899994,133.270004,132.912888,8961500.0,2.0,0.915705,134.927106,...,53.906948,54.096702,6.687990,131.318106,129.596837,18.340000,4.219994,131.804820,142.887970,185.065430


In [20]:
#prediction 업로드시 확인 필요부분


INDICATORS += ['prediction_5','prediction_10','prediction_30']

In [21]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

26613
3591


In [22]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension #현재 상태를 나타내는 state를 정의. 1+ ticker 개수 x2 + 인디케이터 개수 x ticker개수
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 9, State Space: 118


In [28]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}

Earn_list = []

for i in range(10):
  e_train_gym = StockTradingEnv(df = train, **env_kwargs)

  env_train, _ = e_train_gym.get_sb_env()

  if_using_a2c = True ##a2c만 사용해보자

  agent = DRLAgent(env = env_train)
  model_a2c = agent.get_model("a2c")

  if if_using_a2c:
    # set up logger
    tmp_path = RESULTS_DIR + '/a2c'
    new_logger_a2c = configure(tmp_path, ["stdout", "csv", "tensorboard"])
    # Set new logger
    model_a2c.set_logger(new_logger_a2c)
    
  trained_a2c = agent.train_model(model=model_a2c,
                              tb_log_name='a2c',
                              total_timesteps=100000) if if_using_a2c else None
  data_risk_indicator = processed_full[(processed_full.date<TRAIN_END_DATE) & (processed_full.date>=TRAIN_START_DATE)]
  insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])


  e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70,risk_indicator_col='vix', **env_kwargs)
  # env_trade, obs_trade = e_trade_gym.get_sb_env()
  trained_model = trained_a2c
  df_account_value_a2c, df_actions_a2c = DRLAgent.DRL_prediction(
      model=trained_model,
      environment = e_trade_gym)
  
  df_result_a2c = df_account_value_a2c.set_index(df_account_value_a2c.columns[0])
  df_account_value_a2c.to_csv("df_account_value_a2c.csv")
  #baseline stats
  print("==============Get Baseline Stats===========")
  df_dji_ = get_baseline(
          ticker="^DJI",
          start = TRADE_START_DATE,
          end = TRADE_END_DATE)
  stats = backtest_stats(df_dji_, value_col_name = 'close')
  df_dji = pd.DataFrame()
  df_dji['date'] = df_account_value_a2c['date']
  df_dji['account_value'] = df_dji_['close'] / df_dji_['close'][0] * env_kwargs["initial_amount"]
  df_dji.to_csv("df_dji.csv")
  df_dji = df_dji.set_index(df_dji.columns[0])
  df_dji.to_csv("df_dji+.csv")

  result = pd.DataFrame(df_result_a2c)


  result = pd.merge(result, df_dji, left_index=True, right_index=True)


  result.columns = ['a2c','dji']

  print("result: ", result)
  result.to_csv("result.csv")
  
  print('A2C로 얻은 투자 수익률>>', round((df_result_a2c.iloc[-1,0]/df_result_a2c.iloc[0,0]-1)*100,2),'%')
  
  Earn = round((df_result_a2c.iloc[-1,0]/df_result_a2c.iloc[0,0]-1)*100,2)
  
  Earn_list.append(Earn)

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device
Logging to results/a2c
--------------------------------------
| time/                 |            |
|    fps                | 114        |
|    iterations         | 100        |
|    time_elapsed       | 4          |
|    total_timesteps    | 500        |
| train/                |            |
|    entropy_loss       | -12.9      |
|    explained_variance | 0          |
|    learning_rate      | 0.0007     |
|    n_updates          | 99         |
|    policy_loss        | -6.47      |
|    reward             | 0.24104658 |
|    std                | 1.01       |
|    value_loss         | 0.375      |
--------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 114       |
|    iterations         | 200       |
|    time_elapsed       | 8         |
|    total_timesteps    | 1000      |
| train/                |           |
|    entropy

c:\Users\se99a\anaconda3\lib\site-packages\pyfolio\timeseries.py:724: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()


-------------------------------------
| time/                 |           |
|    fps                | 81        |
|    iterations         | 100       |
|    time_elapsed       | 6         |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -12.8     |
|    explained_variance | -0.197    |
|    learning_rate      | 0.0007    |
|    n_updates          | 99        |
|    policy_loss        | -11.9     |
|    reward             | 0.3162262 |
|    std                | 1.01      |
|    value_loss         | 1.39      |
-------------------------------------
--------------------------------------
| time/                 |            |
|    fps                | 81         |
|    iterations         | 200        |
|    time_elapsed       | 12         |
|    total_timesteps    | 1000       |
| train/                |            |
|    entropy_loss       | -12.9      |
|    explained_variance | -0.159     |
|    learning_rate      | 0.0007     |
| 

c:\Users\se99a\anaconda3\lib\site-packages\pyfolio\timeseries.py:724: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()


----------------------------------------
| time/                 |              |
|    fps                | 75           |
|    iterations         | 100          |
|    time_elapsed       | 6            |
|    total_timesteps    | 500          |
| train/                |              |
|    entropy_loss       | -12.8        |
|    explained_variance | 0.502        |
|    learning_rate      | 0.0007       |
|    n_updates          | 99           |
|    policy_loss        | -17.6        |
|    reward             | -0.001757413 |
|    std                | 1.01         |
|    value_loss         | 1.95         |
----------------------------------------
--------------------------------------
| time/                 |            |
|    fps                | 77         |
|    iterations         | 200        |
|    time_elapsed       | 12         |
|    total_timesteps    | 1000       |
| train/                |            |
|    entropy_loss       | -12.9      |
|    explained_variance | 1.19e-

...

c:\Users\se99a\anaconda3\lib\site-packages\pyfolio\timeseries.py:724: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()


-------------------------------------
| time/                 |           |
|    fps                | 73        |
|    iterations         | 100       |
|    time_elapsed       | 6         |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -12.9     |
|    explained_variance | 0.113     |
|    learning_rate      | 0.0007    |
|    n_updates          | 99        |
|    policy_loss        | -20.9     |
|    reward             | 0.0668218 |
|    std                | 1.02      |
|    value_loss         | 3.41      |
-------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 74        |
|    iterations         | 200       |
|    time_elapsed       | 13        |
|    total_timesteps    | 1000      |
| train/                |           |
|    entropy_loss       | -12.9     |
|    explained_variance | 0.22      |
|    learning_rate      | 0.0007    |
|    n_updat

c:\Users\se99a\anaconda3\lib\site-packages\pyfolio\timeseries.py:724: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()


--------------------------------------
| time/                 |            |
|    fps                | 72         |
|    iterations         | 100        |
|    time_elapsed       | 6          |
|    total_timesteps    | 500        |
| train/                |            |
|    entropy_loss       | -12.8      |
|    explained_variance | 0.557      |
|    learning_rate      | 0.0007     |
|    n_updates          | 99         |
|    policy_loss        | -22.9      |
|    reward             | 0.25826782 |
|    std                | 1          |
|    value_loss         | 4.17       |
--------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 75        |
|    iterations         | 200       |
|    time_elapsed       | 13        |
|    total_timesteps    | 1000      |
| train/                |           |
|    entropy_loss       | -12.8     |
|    explained_variance | -0.142    |
|    learning_rate      | 0.0007  

c:\Users\se99a\anaconda3\lib\site-packages\pyfolio\timeseries.py:724: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()


In [29]:
print('평균 수익률>>', round(np.mean(Earn_list),2),'%')
print('표준편차>>', round(np.std(Earn_list),2),'%')

평균 수익률>> 2.38 %
표준편차>> 7.13 %


In [30]:
pd.DataFrame(Earn_list, columns=['수익률'])

,수익률
0,-6.43
1,-10.46
2,2.28
3,-0.51
4,9.06
5,8.84
6,-0.47
7,9.76
8,-0.60
9,12.34
